In [1]:
%load_ext lab_black

In [2]:
from sklearn.cluster import KMeans
import json
import warnings

# warnings.filterwarnings("ignore")
import pandas as pd
from utils import import_df

In [3]:
df = import_df("3-noise-generation-output.json")
df

,s_00,s_01,s_02,s_03,s_04,s_05,s_06,s_07,s_08,s_09,...,vib_y_f3,vib_y_kurt,vib_y_peak,vib_y_vel,vib_z_acc,vib_z_kurt,vib_z_peak,vib_z_vel,current,label
0,0.069769,-5.304294,-9.989233,-3.313071,-2.672482,0.466632,-1.934575,-0.543654,-0.794152,0.040593,...,2.682579,1.049357,0.539067,1.997897,1.440183,0.277093,0.462698,3.196192,3.963271,working-engine-1
1,-0.396607,-7.557986,-6.757172,-4.035399,-1.691554,-1.837123,-0.861492,-0.578646,-0.070114,-1.084699,...,1.089574,0.892152,0.665190,2.834736,2.003717,0.314759,0.469994,2.234707,3.939892,working-engine-1
2,0.843336,-4.064155,-9.813102,-4.427802,-3.048554,-0.725644,-1.227282,0.337728,-0.180079,-0.348343,...,1.738490,0.684112,0.657757,1.623268,1.783158,0.079913,0.598511,2.110562,3.968693,working-engine-1
3,-0.288229,-7.640477,-5.781615,-3.505385,-2.119089,-1.170490,-0.578574,-0.796474,0.391411,0.544625,...,1.079702,0.372437,0.849677,2.632377,1.682203,0.037757,0.722920,1.758257,3.969759,working-engine-1
4,-0.791232,-7.133791,-7.910312,-2.538680,-2.150410,-0.871752,-0.966725,0.189834,-0.712023,-0.627217,...,1.253875,0.658574,0.758675,3.713024,1.625020,0.503656,0.334853,1.777892,3.959930,working-engine-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31795,-0.455047,-1.966358,0.206042,1.096095,-1.075177,0.715201,1.513090,-0.259385,-0.282248,-0.881456,...,0.489770,0.876538,0.848715,0.924194,0.806776,0.880826,1.127457,2.052010,3.947527,working-engine-2
31796,1.057003,0.223313,4.536409,-0.630462,-0.305872,-0.021064,1.774274,-0.613302,-0.132349,-0.251413,...,-0.448210,0.713084,1.246230,1.348640,1.164167,1.341331,0.947828,1.962668,3.951634,working-engine-2
31797,-0.009944,4.708101,3.201700,-2.997003,1.392459,1.217546,0.505933,0.122175,-0.284000,-0.392447,...,1.665808,1.091673,1.133503,1.573948,0.878939,1.169786,1.051456,0.204291,3.955825,working-engine-2
31798,-2.167536,1.077189,-1.609717,-0.211635,0.795443,0.083935,-0.204140,-1.015827,0.817852,0.171630,...,0.293601,1.175417,1.282980,2.269044,0.845278,0.696941,1.119900,1.417757,3.971053,working-engine-2


## Compute Centroid Distance

accepts a pd.DataFrame obj

returns a wider df with the distance from each column to its respective centroid column

n_clusters (default 4) is the number of groups to cluster data into

Added suffix to column "_cd"

In [4]:
n_clusters = 4

# Remove label column
df_num = df.select_dtypes("number")

# kmeans = KMeans(init="k-means++", n_clusters=n_clusters)
# kmeans_fitted = kmeans.fit(df_num)

# retrieve the respective group label for each centroid
# centroids_labels = [
#     kmeans_fitted.predict(pd.DataFrame(centroid).T)[0]
#     for centroid in kmeans_fitted.cluster_centers_
# ]
centroids_labels

[0, 1, 2, 3]

In [9]:
# create a dict with centroid labels as keys and actual centroids as values
centroids_with_labels = dict(zip(centroids_labels, kmeans_fitted.cluster_centers_))
pd.DataFrame(centroids_with_labels).T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,-0.544592,4.334914,-0.690905,-0.211148,0.021703,0.077448,0.272290,-0.171929,-0.150513,-0.045758,...,0.710797,0.692172,1.042222,1.037428,1.450000,1.128713,0.981887,1.074153,1.783432,3.955738
1,0.416482,1.460121,-0.123896,-0.158371,0.028201,0.104173,0.264565,-0.177035,-0.163507,-0.059920,...,0.961542,0.902975,1.034039,1.102276,1.731933,1.007595,0.985483,0.976625,1.236178,3.958491
2,-1.197527,3.048745,0.514906,-0.142977,0.018043,0.142602,0.259862,-0.132130,-0.137593,-0.009359,...,0.892468,0.864071,1.010718,1.007374,1.032962,1.021024,0.985642,0.991324,1.113695,3.945423
3,0.555710,-5.296798,-6.088330,-3.550897,-1.998481,-0.965908,-0.952469,-0.079394,-0.083184,0.011268,...,1.154743,1.023021,0.554981,0.779132,2.874028,1.737525,0.392768,0.636385,2.171854,3.956652


# Computer per-point position

List with length matching the original df with centroids aligned with its respective data points this allows to create a df to subtract and calculate distances

In [6]:
centroids_list = [centroids_with_labels[label] for label in kmeans_fitted.labels_]
centroids_df = pd.DataFrame(centroids_list)
# Rename columns from 0, 1, 2 to s_00, s_01, s_02
centroids_df.columns = df_num.columns
centroids_df

,s_00,s_01,s_02,s_03,s_04,s_05,s_06,s_07,s_08,s_09,...,vib_y_f2,vib_y_f3,vib_y_kurt,vib_y_peak,vib_y_vel,vib_z_acc,vib_z_kurt,vib_z_peak,vib_z_vel,current
0,0.555710,-5.296798,-6.088330,-3.550897,-1.998481,-0.965908,-0.952469,-0.079394,-0.083184,0.011268,...,1.154743,1.023021,0.554981,0.779132,2.874028,1.737525,0.392768,0.636385,2.171854,3.956652
1,0.555710,-5.296798,-6.088330,-3.550897,-1.998481,-0.965908,-0.952469,-0.079394,-0.083184,0.011268,...,1.154743,1.023021,0.554981,0.779132,2.874028,1.737525,0.392768,0.636385,2.171854,3.956652
2,0.555710,-5.296798,-6.088330,-3.550897,-1.998481,-0.965908,-0.952469,-0.079394,-0.083184,0.011268,...,1.154743,1.023021,0.554981,0.779132,2.874028,1.737525,0.392768,0.636385,2.171854,3.956652
3,0.555710,-5.296798,-6.088330,-3.550897,-1.998481,-0.965908,-0.952469,-0.079394,-0.083184,0.011268,...,1.154743,1.023021,0.554981,0.779132,2.874028,1.737525,0.392768,0.636385,2.171854,3.956652
4,0.555710,-5.296798,-6.088330,-3.550897,-1.998481,-0.965908,-0.952469,-0.079394,-0.083184,0.011268,...,1.154743,1.023021,0.554981,0.779132,2.874028,1.737525,0.392768,0.636385,2.171854,3.956652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31795,-1.197527,3.048745,0.514906,-0.142977,0.018043,0.142602,0.259862,-0.132130,-0.137593,-0.009359,...,0.892468,0.864071,1.010718,1.007374,1.032962,1.021024,0.985642,0.991324,1.113695,3.945423
31796,-0.544592,4.334914,-0.690905,-0.211148,0.021703,0.077448,0.272290,-0.171929,-0.150513,-0.045758,...,0.710797,0.692172,1.042222,1.037428,1.450000,1.128713,0.981887,1.074153,1.783432,3.955738
31797,-1.197527,3.048745,0.514906,-0.142977,0.018043,0.142602,0.259862,-0.132130,-0.137593,-0.009359,...,0.892468,0.864071,1.010718,1.007374,1.032962,1.021024,0.985642,0.991324,1.113695,3.945423
31798,-1.197527,3.048745,0.514906,-0.142977,0.018043,0.142602,0.259862,-0.132130,-0.137593,-0.009359,...,0.892468,0.864071,1.010718,1.007374,1.032962,1.021024,0.985642,0.991324,1.113695,3.945423


In [7]:
# calc ditances between original df and centroids df
df_cd = df_num.subtract(centroids_df)
df_cd.columns = [f"{cn}_cd" for cn in df_num.columns]

df_join = pd.concat([df, df_cd], axis=1)
df_join["kmeans_label"] = kmeans.labels_

df_join

,s_00,s_01,s_02,s_03,s_04,s_05,s_06,s_07,s_08,s_09,...,vib_y_f3_cd,vib_y_kurt_cd,vib_y_peak_cd,vib_y_vel_cd,vib_z_acc_cd,vib_z_kurt_cd,vib_z_peak_cd,vib_z_vel_cd,current_cd,kmeans_label
0,0.069769,-5.304294,-9.989233,-3.313071,-2.672482,0.466632,-1.934575,-0.543654,-0.794152,0.040593,...,1.659559,0.494376,-0.240065,-0.876130,-0.297342,-0.115675,-0.173687,1.024337,0.006619,3
1,-0.396607,-7.557986,-6.757172,-4.035399,-1.691554,-1.837123,-0.861492,-0.578646,-0.070114,-1.084699,...,0.066554,0.337171,-0.113943,-0.039292,0.266192,-0.078009,-0.166391,0.062853,-0.016760,3
2,0.843336,-4.064155,-9.813102,-4.427802,-3.048554,-0.725644,-1.227282,0.337728,-0.180079,-0.348343,...,0.715469,0.129131,-0.121375,-1.250759,0.045632,-0.312854,-0.037873,-0.061293,0.012041,3
3,-0.288229,-7.640477,-5.781615,-3.505385,-2.119089,-1.170490,-0.578574,-0.796474,0.391411,0.544625,...,0.056681,-0.182544,0.070545,-0.241651,-0.055322,-0.355011,0.086535,-0.413597,0.013107,3
4,-0.791232,-7.133791,-7.910312,-2.538680,-2.150410,-0.871752,-0.966725,0.189834,-0.712023,-0.627217,...,0.230854,0.103593,-0.020457,0.838996,-0.112506,0.110888,-0.301531,-0.393962,0.003278,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31795,-0.455047,-1.966358,0.206042,1.096095,-1.075177,0.715201,1.513090,-0.259385,-0.282248,-0.881456,...,-0.374300,-0.134180,-0.158659,-0.108767,-0.214248,-0.104816,0.136134,0.938315,0.002104,2
31796,1.057003,0.223313,4.536409,-0.630462,-0.305872,-0.021064,1.774274,-0.613302,-0.132349,-0.251413,...,-1.140383,-0.329138,0.208802,-0.101361,0.035454,0.359444,-0.126325,0.179236,-0.004105,0
31797,-0.009944,4.708101,3.201700,-2.997003,1.392459,1.217546,0.505933,0.122175,-0.284000,-0.392447,...,0.801738,0.080955,0.126129,0.540986,-0.142084,0.184143,0.060132,-0.909404,0.010402,2
31798,-2.167536,1.077189,-1.609717,-0.211635,0.795443,0.083935,-0.204140,-1.015827,0.817852,0.171630,...,-0.570469,0.164699,0.275605,1.236082,-0.175745,-0.288701,0.128576,0.304062,0.025630,2
